# ESP32 Board Design
> Some notes on designing my own esp32 board

- keywords: ['espressif', 'Microprocessor', 'ESP32']

## The ESP32

### Specification

#### General Purpose I/O
- **34 × Programmable GPIOs**

#### Analog & Touch
- **12-bit SAR ADC**: Up to 18 channels
- **2 × 8-bit DAC**
- **10 × Touch Sensors**

#### Communication Interfaces
- **4 × SPI**
- **2 × I²S**
- **2 × I²C**
- **3 × UART**
- **1 × Host (SD/eMMC/SDIO)**
- **1 × Slave (SDIO/SPI)**
- **Ethernet MAC**: Dedicated DMA, IEEE 1588 support
- **CAN 2.0**
- **IR (TX/RX)**

- **Pin 16** – RX2  
- **Pin 17** – TX2  

#### PWM & Sensors
- **Motor PWM**
- **LED PWM**: Up to 16 channels
- **Hall Sensor**

#### Wireless
- **4 × Virtual Wi-Fi Interfaces**
- **Simultaneous Infrastructure BSS Station/SoftAP/Promiscuous Modes**
- **Automatic Beacon Monitoring (hardware TSF)**
- **Multi-connections in Classic BT and BLE**
- **Simultaneous Advertising and Scanning**

#### Internal Memory
- **448 KB ROM**: Booting and core functions
- **520 KB On-chip SRAM**: Data and instructions
- **16 KB SRAM in RTC**
- **8 KB RTC FAST Memory**: Accessible by main CPU during RTC Boot from Deep-sleep
- **8 KB RTC SLOW Memory**: Accessible by co-processor during Deep-sleep

#### Power
- **Operating Voltage**: 2.3 V to 3.6 V
- **Recommended Supply**: 3.3 V, ≥500 mA

#### CPU
- **Xtensa® 32-bit LX6**: Single or dual-core, up to 40 MHz
- **Interfaces**:
    - Xtensa RAM/ROM Interface (instructions/data)
    - Xtensa Local Memory Interface (fast peripheral register access)
    - QSPI (multiple flash/SRAM chips)
- **Oscillators**:
    - Internal 8 MHz (with calibration)
    - External 2–60 MHz crystal (40 MHz for Wi-Fi/BT)
    - External 32 kHz crystal for RTC (with calibration)

#### DAC Channels
- **2 × 8-bit DAC**: Convert digital signals to analog voltage outputs

#### Capacitive Touch Pin Mapping

| Signal | Pin Name  |
|--------|-----------|
| T0     | GPIO4     |
| T1     | GPIO0     |
| T2     | GPIO2     |
| T3     | MTDO      |
| T4     | MTCK      |
| T5     | MTD1      |
| T6     | MTMS      |
| T7     | GPIO27    |
| T8     | 32K_XN    |
| T9     | 32K_XP    |

---

#### Additional Interfaces
- **UART**: Universal Asynchronous Receiver Transmitter
- **I²C**: Inter-Integrated Circuit


### On Microprocessors  

- The ESP32 is based on the **Xtensa LX6** microprocessor, which is neither ARM nor traditional RISC but uses a proprietary architecture by Tensilica. It primarily uses **ASIC** for its microcontroller design, but it can interface with **MEMS** devices like microphones and sensors.

#### MEMS vs ASIC
- **MEMS**: Micro-Electromechanical Systems.
- **ASIC**: Application-Specific Integrated Circuit.

#### ARM (Advanced RISC Machine)
- Created by David Braben and Ian Bell, built on RISC principles.
- ARMv1 (1985): 32-bit, 1-watt chip.  
- **Cortex A**: Designed for phones and laptops. "32-bit is dead as of 2023."s
- **Cortex M**: Microcontrollers, offers 32-bit processing with processors as small as 12k gates.

#### RISC (Reduced Instruction Set Computing)
- Original RISC processor created by David Patterson. 
- RISCv1 (1978) 
- RISC-V (2010): Open-source, OISA (Open Instruction Set Architecture), supports 32-bit and 64-bit.

## Components

### Misc Notes

- JST connectors should be used for all peripherals. Will require a **capacitor** and **resistor** for driving LEDs.
- **Power Switching**: Ensure the design allows safe alternation between **battery** and **wall power**.
- The **ESP32** operates at **3.3V**.
- Development boards typically use an **AMS1117** voltage regulator to step down the **5V USB input** for powering the UART.
- Avoid powering LED strips from the ESP32's 3.3V or 5V pins.
- Do not use USB power from your PC for LED strips.
- Isolate the power between LED strips and the controller.
- Some batches of analog microphones may have quality issues.
- The ESP32 and ESP8266 boards often experience spurious noise on their ADC pins.

#### Bootloader Mode
To enter **bootloader mode**:
1. Connect a **10kΩ resistor** between the **reset button** and **3.3V** to keep it high.
2. When the reset button is pressed, it pulls the line low.
3. To activate bootloader mode:
   - Press **both the reset and boot buttons**.
   - **Release the reset button first**, then release the boot button.

| Concept     | Implementation | Order |
| ----------- | -----------    | -----------    |
| External Antenna     | NEVER! | -----------    |
| OTA   | ADDON. Save for later        | 8MB Flash ESP    |
| On Board Battery + Charger/Stuff   | Save for later. This is a big thing | -----------    |
| V. Regulation   | Check if it's needed for the pilot.    | HT7333 or AMS1117    |
| OnBoard UART   | Never        | -----------    |
| OnBoard Flash/ Boot BTNs | NEVER  | -----------    |
| Micro USB Battery Connector   | Try for pilot      | Standard |
| JST Battery Connector  | Not for pilot |  -----------  |
| JST Peripheral Connectors  | X2 for pilot | 3 pin or 2?    |
| 5V DC In + Cap  | Needed for Lightstrip Tests. Not needed on board. | Standard    |

### LDO – Low Dropout Regulators

The **1117** series generally performs **better than HT73xx**, especially in terms of current handling and voltage tolerance.

#### Datasheets & Resources
- [AMS1117 Datasheet (Advanced Monolithic Systems)](http://www.advanced-monolithic.com/pdf/ds1117.pdf)
- [HT73xx Series Datasheet](https://www.angeladvance.com/HT73xx.pdf)
- [LDO Regulator Overview – YouTube](https://www.youtube.com/watch?v=xQa_YEqaBYs)
- [Mouser: LDO Voltage Regulators](https://www.mouser.com/Semiconductors/Power-Management-ICs/Voltage-Regulators-Voltage-Controllers/LDO-Voltage-Regulators/_/N-5cgac?P=1z0wa2e)

### INMP441 - Omnidirectional Microphone

Offers background noise filtering (or squelch) for the ESP32, which uses a 12-bit ADC.
 
#### INMP441 Notes
- Leave the MAX4466 gain untouched; it works fine out of the box.
- A MAX4466 and INMP401 should average about **1875** on an ESP32, while a MAX9814 averages about **1350** for all gain settings. 
- A microphone module MAX9814 with Electret Microphone (https://www.aliexpress.com/item/32660519341.html) goes for abouts $2

#### Recommended Options
- **Adafruit**: $6.95 - [Electret Microphone Amp - MAX4466](https://www.adafruit.com/product/1063) with Adjustable Gain.
- **Digikey**: $7.95 - [Electret Microphone Amp Audio Evaluation Board - MAX9814](https://www.digikey.com/en/products/detail/adafruit-industries-llc/1713/4990777?s=N4IgjCBcoLQBxVAYygMwIYBsDOBTANCAPZQDa4ArAEwIC6AvvYVWeAOxgDMIDQA).
- **SparkFun**: $6.95 - [MEMS Microphone Breakout - ICS-40180](https://www.sparkfun.com/products/18011).
- **AliExpress**: $1.61 - [I2S MEMS Microphone - INMP441](https://www.aliexpress.com/i/2251832776111658.html?gatewayAdapt=4itemAdapt).

#### Analog vs. I2S Digital Microphones
- **Recommendation**: Use an I2S digital microphone with ESP32 boards, such as INMP441, ICS-43434, or PDM microphones.
- **Analog Input**:
    - Possible but may experience power fluctuation (3.3V) and noise issues.
    - Analog devices are handled by the "ADC1" unit of the ESP32, which can cause problems when connecting other analog devices (e.g., potentiometers) to the same ADC1 unit.
    - Analog microphones like MAX4466 work best when placed very close to the sound source (30-50cm optimal distance).
- **Digital Input**:
    - Digital microphones like INMP441 can pick up sound from several meters away.
    - On ESP32, use I2S microphones such as INMP441, SPH0645, or ICS-43434.

### Battery Charging Modules

* [TP4056 Module](https://www.ebay.com/i/401377749606)
* [TP4056 Charger Board (Banggood)](https://m.banggood.com/ESP32S-ESP32-0_5A-Micro-USB-Charger-Board-18650-Battery-Charging-Shield-without-Battery-p-1395271.html)
* [TP4056 Datasheet](https://dlnmh9ip6v2uc.cloudfront.net/datasheets/Prototyping/TP4056.pdf)
* [TP5100 Datasheet](https://datasheetspdf.com/pdf-file/1299994/TopPower/TP5100/1)
* [MCP73831 Datasheet](https://cdn.sparkfun.com/assets/learn_tutorials/6/9/5/MCP738312.pdf)
* [Simple LiPo Charger Guide](https://thewanderingengineer.com/2016/01/27/simple-lipo-battery-charger-with-the-mcp73831/)

## Manufacturing

- **Gerber File**: For PCB fabrication.
- **BOM (Bill of Materials)**: For assembly and sourcing.

| Item                          | Cost Estimate         |
|-------------------------------|----------------------|
| PCB + Solder + Stencil + Programming Assembly | $42             |
| Fitting (80 units)            | $120 for 10 circuits |s


### Process Overview

* Create the circuit
* Test it in an online simulator
* Build the circuit physically and encase it
* Design and fabricate the PCB (PCBA stage)
* Design the 3D enclosure
* Print the enclosure and verify fit with the PCBA
* Order more PCBs, stencil, and BOM for scale-up

### PCB Layer Stack

* **Outline**
* **Drill Layers** (Top/Bottom)
* **Solder Paste** (Top/Bottom)
* **Silkscreen** (Top/Bottom)
* **Solder Mask** (Top/Bottom)
* **Copper Traces** (Top/Bottom)

### Manufacturing Services

* [JLCPCB](https://jlcpcb.com)
* [Screaming Circuits](https://www.screamingcircuits.com/)
* [MacroFab](https://macrofab.com/digital-manufacturing/)
* [Circuithub](https://circuithub.com/)
* [Dirty PCBs](http://dirtypcbs.com/store/print3d)
* [Seeed Fusion](http://www.seeedstudio.com/fusion_pcb.html)
* [PCBShopper](https://pcbshopper.com/)
* [Elecrow](https://www.elecrow.com/)
* [ALLPCB](https://www.allpcb.com/)


#### JLCPCB Notes

* **Cost Estimation**: PCB and stencil pricing available on JLCPCB's site.
* **Solder Bridges**: Min spacing of 0.254mm between pads. Add note "solder mask bridge is required" during order.
* **Plated Edges/Slots**: Max length of plated slots should be ≤6mm.
* **Mechanical Features**: Slots, cut-outs, millings, and V-cuts must be on the same layer as the board outline.
* **Silkscreen Texts**: Minimum line width 0.15mm; height ≥0.8mm for clarity.
* **Repeat Orders**: Production files from previous orders are reused without changes. Do not add modification notes.
* **Board Outline Items**: V-cuts, slots, millings must be in the board outline layer. Remove unrelated elements to avoid confusion.
* **Remark Field**: Avoid using English notes as it may delay processing.
* **Order Cancellation**: Orders lacking essential layers (e.g., solder mask, drill, or outline) will be auto-cancelled. Specify intentionally omitted layers.
* **Stencil Thickness**: Determined by JLCPCB engineers; not user-selectable.
* **Tooling Holes**: Must appear in the solder paste layer.
* **Required Files for Order**:
  * Gerber files (RS-274x): copper, soldermask, legend, mechanical, SMD paste, carbon, peel-off, via-fill
  * Excellon drill file(s)
  * Gerbers and drill files must be in one archive and include a board outline

### EasyEDA Tips

* Start with a schematic, then convert to PCB layout.
* EasyEDA supports only double-layer designs by default.

#### For single-layer PCB:

1. Layout everything on one side.
2. Ignore other-side pads.
3. Generate Gerber.
4. Add a comment during order indicating it's a single-layer design.

### Silkscreen Overview

* Typically white, used for component IDs, test points, part numbers, warnings, logos, date codes.
* IPC requires manufacturer and lot tracking marks.
* Specify placement/omission of these if needed.

### Stencil vs PCBA

* **Stencil**: Used for manual solder paste application; cost-effective.
* **PCBA**: Requires full BOM and incurs higher cost due to assembly.

## References


#### ESP32 Design & Power Management

* [ESP32 Board Design Guide](https://jiahuang.github.io/2016/10/01/designing-a-esp32-board/)
* [ESP32 Dev Board on Tindie](https://www.tindie.com/products/ddebeer/esp32-dev-board-wifibluetooth/)
* [ESP32 Hardware Guidelines (PDF)](https://www.espressif.com/sites/default/files/documentation/esp32_hardware_design_guidelines_en.pdf)
* [ESP32 General Info](http://esp32.net/)
* [Powering ESP32 from Battery - Reddit](https://www.reddit.com/r/esp32/comments/f102z9/loosing_my_mind_over_powering_the_esp32_from/)
* [Power ESP32 from Solar Panel](https://randomnerdtutorials.com/power-esp32-esp8266-solar-panels-battery-level-monitoring/)
* [Andreas Spiess: Battery & Voltage Regulation](https://www.youtube.com/watch?v=heD1zw3bMhw)
* [ESP32 Power Tips](https://www.youtube.com/watch?v=ffLU7PSuI5k)

#### Community, Schematics, & Additional Resources

- [ESP32 Pinout Reference – Random Nerd Tutorials](https://randomnerdtutorials.com/esp32-pinout-reference-gpios/)  
- [Luca Dentella: ESP32 UART Tutorial](http://www.lucadentella.it/en/2017/11/06/esp32-26-uart/)
- [ESP32 UART Basics – YouTube](https://www.youtube.com/watch?v=Pc99tkwqS4Y)
- [ESP32 UART Example – Input 1](https://www.youtube.com/watch?v=ZR2ob_wYX60)
- [Espressif Serial Connection Guide](https://docs.espressif.com/projects/esp-idf/en/latest/esp32/get-started/establish-serial-connection.html)
- [ESP32 Forum – UART Discussion](https://esp32.com/viewtopic.php?t=658)
* [Arduino Forum Thread](https://forum.arduino.cc/index.php?topic=357747.0)
* [Electronics SE: LiPo Charging](https://electronics.stackexchange.com/questions/220479/charge-li-po-battery-with-tp4056-or-what-else)
* [SparkFun JST Connectors](https://www.sparkfun.com/search/results?term=jst)
* [How to Read a Schematic](https://learn.sparkfun.com/tutorials/how-to-read-a-schematic)
* [ESP32 Coin Cell Board](https://www.cnx-software.com/2019/02/13/esp32-coincell-battery-board/)
* [Ladyada Equipment List](http://www.ladyada.net/library/equipt/)
* [All About Laser Cutters](https://learn.adafruit.com/all-about-laser-cutters) 
* [Adafruit SMT Manufacturing](https://learn.adafruit.com/smt-manufacturing)
* [In-house PCB Etching](http://www.ladyada.net/library/pcb/inhouseetch.html)  
* [EasyEDA Project Page](https://easyeda.com/charleskarpati)
* [EasyEDA Editor](https://easyeda.com/editor?authenticate=force#)
* [EasyEDA Tutorial](https://easyeda.com/editor#id=fbcc60cfb0d247e59f691554a623c02f|e093c929dbea4fbebbdd9a1d95c63ab3|3beaebc196b2425db179dfd6e55c2b23)
* [How to Generate Gerber Files](https://support.jlcpcb.com/article/22-how-to-generate-the-gerber-files)
- [Sound Detector Hookup Guide](https://learn.sparkfun.com/tutorials/sound-detector-hookup-guide)
- [Electret Mic Breakout Board Hookup Guide](https://learn.sparkfun.com/tutorials/electret-mic-breakout-board-hookup-guide/all)
- [Electret Microphone Tutorial](https://www.instructables.com/Electret-microphone/)
- [MEMS Microphone Hookup Guide](https://learn.sparkfun.com/tutorials/mems-microphone-hookup-guide/all)
- [Drums Trigger LED with a Mic (Arduino Forum)](https://forum.arduino.cc/t/drums-trigger-led-with-a-mic/948511/12)

#### Swadge Projects & DFM Guides

- [Swadge 2023 HW](https://github.com/AEFeinstein/Super-2023-Swadge-HW)
- [Swadge 2023 FW](https://github.com/AEFeinstein/Super-2023-Swadge-FW)
- [Swadge 2021 FW Sandbox](https://github.com/cnlohr/Super-2021-Swadge-FW-Sandbox)
- [Swadge 2021 HW](https://github.com/cnlohr/Super-2021-Swadge-HW), [Swadge Devkit](https://github.com/cnlohr/Swadge-Devkit)
- [Swadge Guide](https://github.com/cnlohr/swadgeguide), [KiCad DFM Guide](https://github.com/cnlohr/swadgeguide/blob/master/kicad_for_mfg/README.md)
- [Seeed DFM eBook](https://statics3.seeedstudio.com/fusion/ebook/PCB%20DFM%20V1.1.pdf)
